In [91]:
# import libraries
import urllib.request
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import pgeocode
#!conda install -c conda-forge folium=0.5.0 --yes
import folium
import requests
from pandas.io.json import json_normalize

In [2]:
# Foursquare login
CLIENT_ID = 'PXLZMKWUHM34CRH3OVZS2AL2QHVQFYQWKQ53MH3K04YNGDBV'
CLIENT_SECRET = 'VUFJAB5KL2NPLRCPIAFKQW3FJ15SQAAATZURVXTXWBRD3RCB'
VERSION = '20200501'
LIMIT = 50

In [18]:
# request and read page
url = "https://www.zipcodestogo.com/Orange/CA/"
page = urllib.request.urlopen(url)
soup = BeautifulSoup(page, "lxml")
print(soup.prettify())

<!DOCTYPE html>
<html lang="en">
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <title>
   ZIP Code Search - Orange County California
  </title>
  <meta content="List of zip codes for Orange County California" name="description"/>
  <meta content="city zip codes county state us usa zipcodes by state" name="keywords"/>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <style type="text/css">
   html {
  box-sizing: border-box;
}
*, *:before, *:after {
  box-sizing: inherit;
}

body {
  background-color: #EEEEEE;
  color: #000305;
  font-size: 87.5%;
  font-family: arial, helvetica, sans-serif;
  text-align: left;
}

h1, h2, h3 {
  margin: 0 0;
}

table {
    border-collapse: collapse;
}

table, th, td {
    border: 1px solid #999999;
}

hr {
  height: 1px;
  color: #F5F5F5;
}

#container {
  margin: 0 auto;
  width:100%;
  clear: both;
}

#networkAd {
  display:none;
}

#header {
  width:100%;
  backgrou

In [23]:
# find table on page
wiki_table=soup.find('table')
wiki_table

<table bgcolor="#cccccc" border="0" cellpadding="0" cellspacing="0" width="100%">
<tr>
<td>
<table align="center" border="0" cellpadding="1" cellspacing="1" width="100%">
<tr bgcolor="#FFFFFF">
<td colspan="4">
<div align="center">
<p class="white">Zip Codes for the county of<br/>
<strong><a href="//www.zipcodestogo.com/Orange/CA/">Orange, California</a></strong>
</p>
</div>
</td>
</tr>
<tr bgcolor="#f5f5f5">
<td width="70"><div align="center"><strong>Zip Code</strong></div></td>
<td width="128"><strong>City</strong></td>
<td width="109"><strong>State</strong></td>
<td width="107"><div align="center"><strong>Zip Code Map</strong></div></td>
</tr>
<tr bgcolor="#ffffff">
<td align="center"><a href="//www.zipcodestogo.com/Buena Park/CA/90620/">90620</a></td>
<td>Buena Park</td>
<td><a href="//www.zipcodestogo.com/California/">California</a></td>
<td><div align="center"><a href="http://maps.google.com/maps?oi=map&amp;q=90620">View Map</a></div></td>
</tr>
<tr bgcolor="#FFFFCC">
<td align="

In [50]:
# create empty list
A=[]
B=[]
C=[]

# add each element of table to each list
for row in wiki_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells) == 4:
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))

In [96]:
# create dataframe using lists A, B, C, D
df=pd.DataFrame(A,columns=['ZipCode'])
df['City']=B
df['State']=C
df = df[df.ZipCode != 'Zip Code']
df = df.drop_duplicates(subset='City', keep='first')
df = df.reset_index(drop=True)
df

,ZipCode,City,State
0,90620,Buena Park,California
1,90623,La Palma,California
2,90630,Cypress,California
3,90631,La Habra,California
4,90680,Stanton,California
...,...,...,...
42,92840,Garden Grove,California
43,92856,Orange,California
44,92861,Villa Park,California
45,92870,Placentia,California


In [97]:
# add latitude and longitude for postal codes
nomi = pgeocode.Nominatim('us')
zip_code = []
lat = []
lon = []
zip_code = df['ZipCode']
for i in zip_code:
    lat.append(nomi.query_postal_code(i)['latitude'])
    lon.append(nomi.query_postal_code(i)['longitude'])
df['Latitude'] = lat
df['Longitude'] = lon

df

,ZipCode,City,State,Latitude,Longitude
0,90620,Buena Park,California,33.8439,-118.0080
1,90623,La Palma,California,33.8491,-118.0398
2,90630,Cypress,California,33.8181,-118.0357
3,90631,La Habra,California,33.9331,-117.9493
4,90680,Stanton,California,33.7991,-117.9956
...,...,...,...,...,...
42,92840,Garden Grove,California,33.7869,-117.9273
43,92856,Orange,California,33.7841,-117.8435
44,92861,Villa Park,California,33.8205,-117.8104
45,92870,Placentia,California,33.8744,-117.8543


In [85]:
df = df.dropna()
df

,ZipCode,City,State,Latitude,Longitude
0,90620,Buena Park,California,33.8439,-118.0080
1,90621,Buena Park,California,33.8772,-117.9893
2,90622,Buena Park,California,33.8462,-118.0031
3,90623,La Palma,California,33.8491,-118.0398
4,90624,Buena Park,California,33.8584,-118.0033
...,...,...,...,...,...
144,92871,Placentia,California,33.8829,-117.8557
145,92885,Yorba Linda,California,33.8911,-117.8222
146,92886,Yorba Linda,California,33.9058,-117.7865
147,92887,Yorba Linda,California,33.8841,-117.7304


In [98]:
latitude = 33.787914
longitude = -117.853104

# create map of Orange County using latitude and longitude values
map_oc = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, label in zip(df['Latitude'], df['Longitude'], df['City']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_oc)  
    
map_oc

In [101]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['City', 
                  'City Latitude', 
                  'City Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [102]:
# get nearby venues in OC
oc_venues = getNearbyVenues(names=df['City'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Buena Park
La Palma
Cypress
La Habra
Stanton
Los Alamitos
Seal Beach
Sunset Beach
Surfside
Irvine
Huntington Beach
Laguna Niguel
El Toro
Foothill Ranch
Capistrano Beach
Corona Del Mar
Costa Mesa
Dana Point
Lake Forest
Laguna Woods
East Irvine
Laguna Beach
Laguna Hills
Midway City
Aliso Viejo
Newport Coast
Newport Beach
San Clemente
San Juan Capistrano
Silverado
Trabuco Canyon
Westminster
Rancho Santa Margarita
Mission Viejo
Ladera Ranch
Santa Ana
Fountain Valley
Tustin
Anaheim
Atwood
Brea
Fullerton
Garden Grove
Orange
Villa Park
Placentia
Yorba Linda


In [103]:
oc_venues.head()

,City,City Latitude,City Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Buena Park,33.8439,-118.008,HangTime,33.844821,-118.001565,Theme Park Ride / Attraction
1,Buena Park,33.8439,-118.008,ampm,33.847152,-118.010748,Convenience Store
2,Buena Park,33.8439,-118.008,Timber Mountain Log Ride,33.844928,-118.000649,Theme Park Ride / Attraction
3,Buena Park,33.8439,-118.008,Charles M. Schulz Theatre at Knott's Berry Farm,33.843509,-118.001522,Theme Park
4,Buena Park,33.8439,-118.008,Silver Bullet,33.844580,-117.999975,Theme Park Ride / Attraction


In [104]:
oc_venues.groupby('City').count()

,City Latitude,City Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
City,,,,,,
Aliso Viejo,50,50,50,50,50,50
Anaheim,21,21,21,21,21,21
Atwood,9,9,9,9,9,9
Brea,25,25,25,25,25,25
Buena Park,50,50,50,50,50,50
...,...,...,...,...,...,...
Trabuco Canyon,4,4,4,4,4,4
Tustin,50,50,50,50,50,50
Villa Park,4,4,4,4,4,4


In [105]:
# one hot encoding
oc_onehot = pd.get_dummies(oc_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
oc_onehot['City'] = oc_venues['City'] 

# move neighborhood column to the first column
fixed_columns = [oc_onehot.columns[-1]] + list(oc_onehot.columns[:-1])
toronto_onehot = oc_onehot[fixed_columns]

oc_onehot.head()

,ATM,Accessories Store,Airport,American Restaurant,Antique Shop,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,...,Video Store,Vietnamese Restaurant,Volleyball Court,Warehouse Store,Weight Loss Center,Wine Bar,Wings Joint,Women's Store,Yoga Studio,City
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Buena Park
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Buena Park
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Buena Park
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Buena Park
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Buena Park


In [106]:
oc_grouped = oc_onehot.groupby('City').mean().reset_index()
oc_grouped

,City,ATM,Accessories Store,Airport,American Restaurant,Antique Shop,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,...,Video Game Store,Video Store,Vietnamese Restaurant,Volleyball Court,Warehouse Store,Weight Loss Center,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Aliso Viejo,0.0,0.0,0.0,0.080,0.0,0.0,0.0,0.0,0.0,...,0.02,0.0,0.000000,0.0,0.0,0.0,0.000,0.0,0.00,0.000
1,Anaheim,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.000000,0.0,0.0,0.0,0.000,0.0,0.00,0.000
2,Atwood,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.000000,0.0,0.0,0.0,0.000,0.0,0.00,0.000
3,Brea,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.000000,0.0,0.0,0.0,0.000,0.0,0.04,0.000
4,Buena Park,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.000000,0.0,0.0,0.0,0.020,0.0,0.00,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42,Trabuco Canyon,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.000000,0.0,0.0,0.0,0.000,0.0,0.00,0.000
43,Tustin,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.040000,0.0,0.0,0.0,0.000,0.0,0.00,0.000
44,Villa Park,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.000000,0.0,0.0,0.0,0.000,0.0,0.00,0.000
45,Westminster,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.028571,0.0,0.0,0.0,0.000,0.0,0.00,0.000


In [107]:
num_top_venues = 5

for city in oc_grouped['City']:
    print("----"+city+"----")
    temp = oc_grouped[oc_grouped['City'] == city].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Aliso Viejo----
                 venue  freq
0  American Restaurant  0.08
1        Grocery Store  0.08
2          Coffee Shop  0.08
3          Pizza Place  0.06
4       Sandwich Place  0.04


----Anaheim----
                           venue  freq
0           Fast Food Restaurant  0.10
1                           Park  0.10
2       Mediterranean Restaurant  0.05
3  Paper / Office Supplies Store  0.05
4                 Baseball Field  0.05


----Atwood----
                  venue  freq
0  Fast Food Restaurant  0.22
1        Sandwich Place  0.11
2    Mexican Restaurant  0.11
3              Pharmacy  0.11
4      Business Service  0.11


----Brea----
            venue  freq
0   Grocery Store  0.08
1      Steakhouse  0.08
2     Pizza Place  0.08
3  Sandwich Place  0.08
4  Cosmetics Shop  0.04


----Buena Park----
                          venue  freq
0  Theme Park Ride / Attraction  0.34
1                    Theme Park  0.18
2             Convenience Store  0.06
3         Performing Arts

In [108]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [117]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['City']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
city_venues_sorted = pd.DataFrame(columns=columns)
city_venues_sorted['City'] = oc_grouped['City']

for ind in np.arange(oc_grouped.shape[0]):
    city_venues_sorted.iloc[ind, 1:] = return_most_common_venues(oc_grouped.iloc[ind, :], num_top_venues)

city_venues_sorted.head()


,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Aliso Viejo,Grocery Store,American Restaurant,Coffee Shop,Pizza Place,Sushi Restaurant,Chinese Restaurant,Pharmacy,Cosmetics Shop,Sandwich Place,Seafood Restaurant
1,Anaheim,Park,Fast Food Restaurant,Home Service,Spa,Mediterranean Restaurant,Skate Park,Baseball Field,Cafeteria,Theater,Locksmith
2,Atwood,Fast Food Restaurant,Convenience Store,Sandwich Place,Business Service,Stadium,Carpet Store,Mexican Restaurant,Pharmacy,Farmers Market,Financial or Legal Service
3,Brea,Sandwich Place,Pizza Place,Steakhouse,Grocery Store,Café,Sushi Restaurant,Basketball Stadium,Smoke Shop,Buffet,Mexican Restaurant
4,Buena Park,Theme Park Ride / Attraction,Theme Park,Convenience Store,Fried Chicken Joint,Coffee Shop,Fast Food Restaurant,Mexican Restaurant,Shopping Mall,Pharmacy,Market


In [118]:
city_venues_sorted.set_index('City', inplace=True)

In [119]:
city_venues_sorted[city_venues_sorted.isin(['Brewery'])].dropna(how='all')

,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
City,,,,,,,,,,
Silverado,Brewery,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
